In [1]:
import DataConnection as Dc
from collections import defaultdict
import time
import datetime
import operator
import csv



pgo = Dc.PGConnection('CRASHDATA', 'shostetter')  #Database Connection
dbo = Dc.DBConnection('dotgissql01', 'gisgrid', 'GISUSER', 'GISUSER') #Database Connection
dbo2 = Dc.DBConnection('DOTDEV55SQL02', 'DataWarehouse', 'SHostetter', 'shostetter') #Database Connection



def get_intersection_universe(pg):
    cur = pg.conn.cursor()
    #.conn.cursor() Handles the connection to a PostgreSQL database instance. It encapsulates a database session. Return a new cursor object using the connection.

    cur.execute("select nodeid, masterid, is_int  from node where is_int = true")#Execute a database operation (query or command)
    # dictionary of nodeid: [masterid, intersection]
    nodes = defaultdict(list, ((int(row[0]), [row[1], row[2]]) for row in cur.fetchall()))
    # cur.fetchall() Fetch all (remaining) rows of a query result, returning them as a list of tuples. An empty list is returned if there is no more record to fetch.
    del cur
    #deletes cur to reset the memory for a new query
    return nodes

def alt_get_signalized_int(db, pg, nodes):
    cur = db.conn.cursor()
    cur.execute("""SELECT [PSGM_ID],[ContrType],[NodeID],[Longitude],[Latitude]
                    FROM [GISGRID].[gisadmin].[SIGNAL_CONTROLLER]
                    --[LION_CURRENT].[GISADMIN].[SIG_TRpsgm] 
                    where NormalizedType != 'Z'
                    --and [Longitude] not in ('','S' ) and  [Latitude] not in ('')
                    and [Longitude] is not null
                    and psgm_id in (41084, 41083)""")
    for row in cur .fetchall():
        psgm_id, contrtype, nodeid, x, y = row
        #print('****',psgm_id, contrtype, nodeid, x, y,'*********')
        nodeid = alt_get_node_from_signal_coords(pg, str(x), str(y))
        #print('-----------',nodeid,'--------------')
              
        if nodeid:
            nodes[int(nodeid)].append('Signal')
    del cur
    return nodes


def alt_get_node_from_signal_coords(pg, x, y):
    cur = pg.conn.cursor()
    cur.execute("""select nodeid, st_distance(geom, st_transform(st_geomfromtext(
                        'SRID=4326;POINT(%s %s)'), 2263)) as distance
                    from node where st_distance(geom, st_transform(st_geomfromtext(
                        'SRID=4326;POINT(%s %s)'), 2263)) < 1000
                    and is_int = true
                    order by geom <#> st_transform(st_geomfromtext('SRID=4326;POINT(%s %s)'), 2263) limit 1
                """ % (x, y, x, y, x, y)
                )
    row = cur.fetchone()
    del cur
    if row:
        return int(row[0])
    else:
        return None
# make list of unsignalized masterids

def get_unsignalized_masterids(nodes):
    mids = defaultdict(list)
    to_remove = set()
    for node in nodes.keys():
        mids[nodes[node][0]].append(node)
        if nodes[node][-1] == 'Signal':
            to_remove.add(nodes[node][0])
    for m in to_remove:
        del(mids[m])
    return mids

········

































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [2]:
node_dict = get_intersection_universe(pgo)

In [3]:
node_dict = alt_get_signalized_int(dbo, pgo, node_dict)

In [4]:
masters = get_unsignalized_masterids(node_dict)

In [5]:
masters

defaultdict(list,
            {9043971: [9043971, 44038],
             14: [14],
             18: [18],
             19: [19],
             22: [22],
             23: [23],
             25: [25],
             26: [26],
             27: [27],
             9044000: [9043999, 9044000, 53354, 101411, 101412],
             9044003: [9044003, 3830],
             9044004: [9044004],
             47: [47],
             51: [51],
             52: [52],
             53: [53],
             54: [54],
             55: [55],
             56: [56],
             57: [57],
             59: [59],
             62: [62],
             63: [63],
             64: [64],
             65: [65],
             66: [66],
             67: [67],
             68: [68],
             69: [69],
             70: [70],
             71: [71],
             72: [72],
             74: [74],
             75: [75],
             76: [76],
             77: [77],
             78: [78],
             79: [79],
             80: [80],


In [22]:
def get_all_crashes_nypd(db2):
    # test speed of doing 1 query and iterating in python memory
    cur = db2.conn.cursor()
    cur.execute( """SELECT distinct c.[ANUM_PCT]+c.[ANUM_YY]+c.[ANUM_SEQ] as crashid ,
                                    c.[NODEID], cast(c.[OCCURRENCE_DATETIME] as DATE) as accd_date,
                                    c.[INJURED_COUNT],c.[KILLED_COUNT],  
                                    case when vic.ped_nonped = 'P' then 1
                                    when vic.PED_NONPED = 'B' then 2 else 3 end as inj_mode,
                                    c.[ACCIDENT_DIAGRAM]  AS [COLLISION_TYP],
                                    null as ext_of_inj, 0 as pre_action, 0 as direction, 0 as contrib
                    FROM [DataWarehouse].[dbo].[AIS_PD_Core_F] AS c
                    LEFT OUTER JOIN (
                            --selects crashid breakdown, and non preventable accident actions 
                            SELECT [ANUM_PCT], [ANUM_YY], [ANUM_SEQ],
                                    MAX(CASE WHEN PRE_ACDNT_ACTION = '10' THEN 1
                                             WHEN PRE_ACDNT_ACTION in ('15', '08', '09', '14', '04') THEN 1 -- removes other non-preventable
                                             ELSE 0 END) AS PK
                            FROM [DataWarehouse].dbo.AIS_PD_Vehicle_F
                            GROUP BY [ANUM_PCT], [ANUM_YY], [ANUM_SEQ]
                            )AS v -- parked vehicles
                            ON c.[ANUM_PCT] = v.ANUM_PCT AND c.[ANUM_YY] = v.ANUM_YY AND c.[ANUM_SEQ] = v.ANUM_SEQ
                    LEFT OUTER JOIN DataWarehouse.dbo.AIS_PD_Pedestrian_F AS P	 -- ??	
                            ON c.[ANUM_PCT] = P.ANUM_PCT AND c.[ANUM_YY] = P.ANUM_YY AND c.[ANUM_SEQ] = P.ANUM_SEQ
                    LEFT OUTER JOIN DataWarehouse.dbo.AIS_PD_Victim_F as vic --Victim Table
                            ON c.ANUM_PCT = vic.ANUM_PCT and c.ANUM_SEQ = vic.ANUM_SEQ and c.ANUM_YY = vic.ANUM_YY
                    WHERE C.OCCURRENCE_DATETIME > (select dateadd(day, -7, dateadd(month, -7, cast('10/20/2016' as datetime)))) 
                    and C.OCCURRENCE_DATETIME < cast('10/20/2016' as datetime)
                    --and C.NODEID != 0
                    and v.PK != 1 
                    and c.NODEID is not null 
                """)
    data = cur.fetchall()
    del cur
    # clean up dates
    for row in data:
        d = row[2]  # date field
        row[2] = datetime.datetime.strptime(d, "%Y-%m-%d").date()
    assert isinstance(data, object)
    return data

In [23]:
crash_list = get_all_crashes_nypd(dbo2)  # NYPD data
crash_list

[(u'0011600576', 20516, datetime.date(2016, 3, 13), 0, 0, 3, u'9', None, 0, 0, 0),
 (u'0011600577', 20240, datetime.date(2016, 3, 13), 0, 0, 3, u'9', None, 0, 0, 0),
 (u'0011610002', 15202, datetime.date(2016, 3, 13), 0, 0, 3, u'3', None, 0, 0, 0),
 (u'0011610004', 15429, datetime.date(2016, 3, 14), 0, 0, 3, u'2', None, 0, 0, 0),
 (u'0011610007', 20087, datetime.date(2016, 3, 14), 0, 0, 3, u'1', None, 0, 0, 0),
 (u'0011610008', 15283, datetime.date(2016, 3, 14), 1, 0, 1, u'9', None, 0, 0, 0),
 (u'0011610009', 20369, datetime.date(2016, 3, 14), 0, 0, 3, u'2', None, 0, 0, 0),
 (u'0011610010', 15200, datetime.date(2016, 3, 14), 0, 0, 3, u'5', None, 0, 0, 0),
 (u'0011610011', 15371, datetime.date(2016, 3, 15), 0, 0, 3, u'4', None, 0, 0, 0),
 (u'0011610012', 20222, datetime.date(2016, 3, 14), 0, 0, 3, u'8', None, 0, 0, 0),
 (u'0011610016', 9030985, datetime.date(2016, 3, 15), 0, 0, 3, u'2', None, 0, 0, 0),
 (u'0011610017', 15180, datetime.date(2016, 3, 14), 0, 0, 3, u'1', None, 0, 0, 0),
 (

In [8]:
def get_all_dir_right_angle_crashes_nypd(db2):
    cur = db2.conn.cursor()
    cur.execute("""/* conservative */
                    -- north / south
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (1, 5)
                        and PRE_ACDNT_ACTION != '10'
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (3, 7)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- east / west
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (3,7)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (1, 5)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- northeast / southwest
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (2, 6)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (4, 8)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- northwest / southeast
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (4, 8)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (2, 6)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER""")
    ra = cur.fetchall()
    del cur
    return set([i[0] for i in ra])

In [9]:
drac = get_all_dir_right_angle_crashes_nypd(dbo2)  # NYPD data

In [10]:
for i in drac: print(int(i))

431702557
431702556
1231610263
521703263
1221700085
471703062
731611031
131701166
1211501863
1231800286
661502918
471703069
431702558
1051705931
61700629
401501429
1151702803
1111702063
1041801232
1201600009
1081802371
1151701712
1141700915
61800492
61800493
1141703999
191610964
421701104
661502913
491610119
181503926
191610963
1051801717
1011600056
1081701185
491800436
1141701848
1211800202
1041613421
1061600243
1061800241
1061600241
71610820
1211801156
1111502957
1211801152
61800728
141503683
1151612761
241501169
431501973
141503684
191700191
661800274
661702131
661702132
941611311
171801610
671610830
1041701595
1201610900
1131501909
1091705235
1111502953
701612115
421500804
1081704980
711611296
1001610284
701610984
1091706393
701610982
431700349
521610373
1131610815
1201702239
521610378
621610732
621610733
1061600797
1061600790
471611339
1221802056
71700410
181701633
471611336
191700227
731610341
431703606
841502781
431703601
71500756
1071611317
751610681
621800491
1071611313
721612

471800326
491801431
491701809
1051614828
491701807
491701804
1091702824
1101702992
1051705535
1001610039
1131610012
621600390
1051705532
1081600525
1011800675
1011800676
1081600523
51801124
1051505422
51801120
1031702137
1051505427
671703910
1031702138
1091702821
621701459
611701517
1231501277
1041503446
71701063
1221611652
11701167
611701518
1041502071
621612059
1041502072
1131612042
1091707382
831700572
621611883
621611882
261610709
261610708
831700571
1211802255
1231700046
1141702210
101700514
671610357
731801569
1201600433
441611945
1091704388
441611940
1131612609
811800369
831502051
1131610015
491702385
1121610813
1051803412
1091703201
1091703200
901701959
901701957
751600582
441610097
1051800430
1081504492
1131703563
1021800290
1131703561
1071700111
1041611961
1061501936
1101502218
751612940
1201702355
1051611644
1021702150
611702157
1051706343
621611777
621611776
621611770
621800166
771800914
681801954
621800169
1021703206
661611646
1081701864
1021703202
661611642
781700175
1717

521600457
1211503063
901800287
631610222
1201701560
1201701567
771610397
11611093
341700570
631610229
471801194
11611094
751600288
751613828
1081801686
251700180
751610571
681502593
751613822
1231501500
341600250
771611415
751613825
1051802534
1051802537
691702089
1081613999
631700609
721702522
1211600705
251701850
1081612375
631700601
1211600703
401802184
1061700989
481800279
1041503257
491610837
281700910
1091610821
281700916
1061700987
1061700984
1091610825
611610259
1011701134
761800192
1051504396
191600518
1011701047
1091613490
1061610858
1061610859
671702725
671801012
1151600538
521502563
1051600470
1061610857
1061800003
1041702375
1081613664
1111704071
771611518
1131700508
471611860
471611861
471611864
471611866
1001610249
1091707235
1061611109
1231611059
1061611106
1061611104
1061611103
781600054
1011701136
451611259
1101611358
1041701413
1211503668
1201611563
451611255
101701936
1021613273
711611607
441610357
441610351
721800826
441610353
701610726
1231800476
881800131
1021613

491501994
491501995
1101800267
1061701655
621610890
1121702816
1141611076
1201700086
451611040
1051610346
451611043
1051610343
661610416
321600114
671703880
671703882
621702014
1051801188
621703084
901801874
341701054
1061802257
471600066
471703815
471703814
481700562
411701781
191611802
1211611412
631701725
501700844
1211802067
1221611654
471801838
1031611524
631701728
1141702434
941702541
1111612138
1131702686
1031800501
341701586
1061800372
631702193
631702192
631702194
691610473
901612245
521502314
701703421
691610478
901612240
751614085
761611208
701801314
1011610231
701801310
1031702638
101702271
1041503194
1091612658
471612200
1231501269
1101600466
1031800245
1211502367
181613322
1061802481
1131701840
751613659
1141610308
901501694
471503369
1091610606
691610789
1011700292
781610026
471503360
1141610307
1101610231
1131612204
1131612202
1131612201
191801706
721702171
721702170
1211600359
731702606
691610884
1041802534
1011500784
1011500782
1011500781
721502205
721502204
901703565

471600615
751702157
1081600180
471600611
1071701135
1021501989
941702118
1021612418
841611530
1141612447
11502457
621800365
411701119
301800542
601801312
1111702545
661611081
331800424
1031610437
431501601
411610855
411610857
1221610188
411610853
1091700973
661701352
521702207
1091600091
461702062
691800907
1051600495
691800908
431703912
431703913
471701826
1091706780
471701820
811500663
1091506280
601700335
1051706284
811500665
171801516
171801517
1211701023
941611640
1051610769
301610507
51800143
751802267
1151701651
771700287
1221610184
1021801991
1021502869
191612125
1211611763
1141803038
1141701194
1071700083
1141701196
1061700663
721702722
1061502101
1131613189
841702197
331500915
191611477
751504604
1101701679
1061700660
691800577
61800390
751503466
1061801418
751503464
141503170
491501892
1221702871
191505254
811600025
811600026
1061700665
251702468
771700750
681610407
1111703633
1061700664
771700756
1201612102
791610988
1091614345
1201612101
1091614341
1211503157
601800957
621

...

941610910
461610348
171701474
461610346
1091600321
1061612749
1091600329
1001610093
1111612325
661610338
1031612108
1131800996
1131800994
501801385
141800381
1131800993
1081704291
1021503165
1081704292
1131800999
751614440
1101801214
401701253
441502215
481702317
1061600674
411701891
471501747
421501430
191613277
1131802176
621801031
321700773
11701470
321700775
431503269
941701046
621801030
1221612646
431503267
251611918
1051615239
1101800645
751702538
681612427
471503455
471503452
901600349
901600348
901600347
1091801904
141502203
1201501518
1091801900
841600227
1051613961
131800281
251610354
1071612121
451700122
131800289
621800786
1071612128
1151600257
261610256
421610700
1091804373
171612903
1041802199
171612900
421610708
1041802190
451701836
201701854
1131701948
831801061
251800431
1131801406
1091700594
1091700596
751600987
701702982
731611924
731611925
631800526
1021800198
631800522
631800521
711801145
831800815
691600106
721503300
1131611617
1021800192
1221700953
621800257
6616

71700704
751803749
771600174
51701598
661600672
71500624
51701596
661600671
131700366
191610495
1051503994
141612704
141700459
191610493
1091800115
1071610772
421700748
1131701639
491611733
1061611407
831611177
181700005
1091800116
421700747
751613465
681700164
451501952
1151801606
451501956
841612630
1131703388
791701295
1061600335
431502073
1061600330
1131800952
731610183
1201501311
781800617
331701418
751504867
751610842
1201610693
751504862
1151800559
791610485
131700092
761611598
681800428
1151801367
1131611444
1021701477
1041612006
1021701471
621610768
691701698
1071703650
1071703652
261700746
1131801012
411600347
1091705000
751613797
1101700499
751613791
1141703532
1131702464
1061600480
1221610139
1211801791
1011610889
521611511
1221802124
1221610131
471611423
1221802121
1011610881
1011610882
471611427
1061502942
901801162
1101801946
1141612965
731610507
1091600551
941501322
1031700610
191703626
721503291
791611507
791611504
171502539
671611741
1231611000
341501071
1201611876
60

1081802793
231701363
901801538
771801002
231701360
601701204
1051612284
1031600436
1051612286
91500754
1131503145
1131701722
1121801004
1131701721
1091702603
241700117
1091705576
751611275
1021802183
1131701727
1061701991
1061612760
791800597
671801671
681800580
1031611182
1131703238
1051600076
681800587
341701820
681800589
881611439
1221703695
1051601033
881611436
731800048
631701258
241610136
1051601036
1051601035
1051601034
421700714
461611315
231800346
631701255
1121503109
881611435
431800507
601701984
1021802239
721700713
1051611399
751613437
1021802231
1051701488
671704034
671704035
1031503001
671703025
1021800368
1021611636
1051505127
1021611638
941801261
1081802155
1081610100
1031701228
881610474
831800718
101801669
441702462
1211703486
1031701225
831800711
1031701027
681801189
1041600249
751802654
1071504010
1041503943
1061600307
1051611647
1051611646
1041503947
1051611643
1051611642
1201701901
1091803731
1141801361
1061600305
711610669
1091611975
671800069
1051600902
32170003

791611628
631611658
1031700558
631610478
491800372
631600507
601501587
491800374
441600372
1141612371
731700765
1091700129
601501589
1131802326
1051802687
1091613070
1211703564
1211503876
1091611774
781502146
1031610588
1061504008
171801062
1021700603
1141613703
1021700607
461611631
1201801382
1201801380
451702890
1101800233
1051701382
1201801388
1141801373
661612561
1041800602
1091611880
1041800606
1041800604
681612257
901801518
1031701377
411801045
1071801304
791801070
1201611358
1221703403
691801199
471700337
791801076
181611763
1041703559
1081700065
1101612907
411700019
1021610117
191614127
281700667
601500987
771801282
1111701461
1041704254
171503161
131801404
171503167
1021701361
771611669
1121701802
1221610215
1091702012
711611863
1101800095
1011700564
711611867
711611866
1121502420
11703013
1021801555
1091611776
1071613058
1071703394
631612302
631612306
1151503046
1211611825
751610268
1051700381
751705355
1111800154
491701315
1051700385
401702672
1091707472
1061802504
109170747

1001800054
441610789
491702497
471613000
471613001
491702492
1151702853
1091702946
701700007
701612803
881610336
1031701749
881610335
881610331
1031701744
1231800023
341501460
621611065
721611249
621611063
1061702068
721611240
11800863
1061702063
621611068
301800271
1021703513
901703020
611701100
781700661
661800895
611701109
901703029
721600349
1231501267
791700577
1041610262
751600472
471702334
751600474
1041700060
1091702940
811700251
711800925
811700256


In [11]:
count=[]
for i in drac:
    if (int(i)) in [1061610117,1061610251,1061610286,1061610293,1061610326,1061610334,1061610343,1061610389,1061610431,
           1061610523,1061610582,1061610741,1061610840,1061610948,1061611237,1061611268,1061612131,
           1061612261,1061612291, 1061612306,1061612384,1061612537,1061612652]:
            
            count.append(i)
            
print(count)

[u'1061610741', u'1061612291', u'1061610293', u'1061612306', u'1061612652', u'1061610582']


In [12]:
class Intersection:
    def __init__(self, masterid, nodeid_list, crashes_list):
        self.masterid = masterid
        self.nodeid_list = nodeid_list
        self.crashes = crashes_list
        self.one_year_crashes = dict()
        self.twelve_month_period()
        self.warrant()
        self.get_12_month_crashes()

    def five_or_more(self):
        count = 0
        for c in self.crashes:
            if c.preventable():
                count += 1
        if count > 4:
            return True
        else:
            return False

    def twelve_month_period(self):
        cl = [[c.crashid, c.ac_date] for c in self.crashes if c.preventable()]
        cl = sorted(cl, key=operator.itemgetter(1), reverse=True)
        while len(cl) > 1:
            if (cl[0][1] - cl[-1][1]).days / 365.0 > 1:
                cl.pop()
            else:
                break
        return cl

    def warrant(self):
        if len(self.twelve_month_period()) > 4:
            return True
        else:
            return False

    def get_12_month_crashes(self):
        crashes = dict((c[0], None) for c in self.twelve_month_period())
        for crash in self.crashes:
            if crash.crashid in crashes.keys():
                crashes[crash.crashid] = crash
        return crashes


In [13]:
def add_intersection_objects_to_master_dict(mids):
    for m in mids:
        mids[m].append(Intersection(m, mids[m], list()))
        if isinstance(mids[m][-1].nodeid_list[-1], Intersection):
            mids[m][-1].nodeid_list = mids[m][-1].nodeid_list[:-1]
            


In [14]:
add_intersection_objects_to_master_dict(masters)

In [15]:
class Crash:
    def __init__(self, crashid, nodeid, ac_date, inj, fat, mode, col_typ, sev, pre_actions, directions, contrib_fac):
        """
        :param crashid:
        :param nodeid:
        :param ac_date:
        :param inj:
        :param fat:
        :param mode:
        :param col_typ:
        :param sev:
        :param pre_actions:
        :param directions:
        :param contrib_fac:
        :return:
        """
        self.crashid = crashid
        self.nodeid = nodeid
        self.inj = inj
        self.fat = fat
        self.mode = mode
        self.col_typ = str(col_typ)
        self.pre_actions = pre_actions
        self.directions = directions
        self.ac_date = ac_date
        self.sev = sev
        self.contib_fac = contrib_fac
        self.right_angle()
        self.ped_bike()
        self.preventable()

    def right_angle(self):
        """
        includes: right angle (4), left turn with (0), and right turn with (5)
        :return: Boolean
        """
        if self.col_typ in ('4', '0', '5'):  # NYPD
            return True
        elif self.col_typ in ('04', '10', '05'):  # NYSDOT
            return True
        else:
            return False

    def ped_bike(self):
        if self.mode in (1, 2):
            return True
        else:
            return False

    def directions_prevent(self):
        if self.directions:
            return True
        else:
            return False

    def preventable(self):
        if self.right_angle() or self.ped_bike() or self.directions_prevent():
            if self.directions_prevent() and not self.right_angle():
                print '%s RA (%s), PB (%s), DIR(%s)' % (str(self.crashid),
                                                    str(self.right_angle()),
                                                    str(self.ped_bike()),
                                                    str(self.directions_prevent())
                                                    )
            return True
        else:
            return False

In [16]:
def add_crashes_to_intersections(nodes, mids, crashes, drac):
    for crash in crashes:
        crashid, nodeid, ac_date, inj, fat, mode, col_typ, sev, pre_actions, directions, contrib_fac = crash
        if crashid in drac:
            directions = True
        else:
            directions = False
        if nodes[int(nodeid)]:
            master = nodes[int(nodeid)][0]
            if master in mids.keys():  # check for unsignalized
                mids[master][-1].crashes.append(Crash(crashid, nodeid, ac_date, inj, fat, mode,
                                                      col_typ, sev, pre_actions, directions, contrib_fac))

In [24]:
add_crashes_to_intersections(node_dict, masters, crash_list, drac)


0011610002 RA (False), PB (False), DIR(True)
0011610053 RA (False), PB (False), DIR(True)
0011610139 RA (False), PB (False), DIR(True)
0011610158 RA (False), PB (False), DIR(True)
0011610207 RA (False), PB (False), DIR(True)
0011610254 RA (False), PB (False), DIR(True)
0011610310 RA (False), PB (False), DIR(True)
0011610405 RA (False), PB (False), DIR(True)
0011610413 RA (False), PB (False), DIR(True)
0011610468 RA (False), PB (False), DIR(True)
0011610474 RA (False), PB (True), DIR(True)
0011610559 RA (False), PB (False), DIR(True)
0011610605 RA (False), PB (False), DIR(True)
0011610759 RA (False), PB (True), DIR(True)
0011610933 RA (False), PB (False), DIR(True)
0011610993 RA (False), PB (False), DIR(True)
0011610998 RA (False), PB (False), DIR(True)
0011611000 RA (False), PB (False), DIR(True)
0011611070 RA (False), PB (False), DIR(True)
0011611076 RA (False), PB (False), DIR(True)
0011611093 RA (False), PB (True), DIR(True)
0011611094 RA (False), PB (False), DIR(True)
0011611175 RA

0131610549 RA (False), PB (True), DIR(True)
0131610597 RA (False), PB (False), DIR(True)
0131610631 RA (False), PB (False), DIR(True)
0131610637 RA (False), PB (False), DIR(True)
0131610650 RA (False), PB (False), DIR(True)
0131610738 RA (False), PB (False), DIR(True)
0131610763 RA (False), PB (True), DIR(True)
0131610792 RA (False), PB (False), DIR(True)
0131610802 RA (False), PB (False), DIR(True)
0131610869 RA (False), PB (False), DIR(True)
0131610895 RA (False), PB (False), DIR(True)
0131610919 RA (False), PB (False), DIR(True)
0131610920 RA (False), PB (False), DIR(True)
0131611029 RA (False), PB (False), DIR(True)
0131611067 RA (False), PB (False), DIR(True)
0131611071 RA (False), PB (False), DIR(True)
0131611086 RA (False), PB (True), DIR(True)
0131611202 RA (False), PB (False), DIR(True)
0131611327 RA (False), PB (True), DIR(True)
0131611583 RA (False), PB (False), DIR(True)
0131611600 RA (False), PB (False), DIR(True)
0131611639 RA (False), PB (False), DIR(True)
0131611644 RA 

0191612722 RA (False), PB (False), DIR(True)
0191612779 RA (False), PB (False), DIR(True)
0191612813 RA (False), PB (False), DIR(True)
0191612931 RA (False), PB (False), DIR(True)
0191612960 RA (False), PB (False), DIR(True)
0191613021 RA (False), PB (False), DIR(True)
0191613052 RA (False), PB (True), DIR(True)
0191613053 RA (False), PB (False), DIR(True)
0191613102 RA (False), PB (False), DIR(True)
0191613147 RA (False), PB (True), DIR(True)
0191613175 RA (False), PB (True), DIR(True)
0191613175 RA (False), PB (False), DIR(True)
0191613290 RA (False), PB (False), DIR(True)
0191613368 RA (False), PB (False), DIR(True)
0201600352 RA (False), PB (False), DIR(True)
0201610117 RA (False), PB (False), DIR(True)
0201610145 RA (False), PB (False), DIR(True)
0201610148 RA (False), PB (False), DIR(True)
0201610150 RA (False), PB (False), DIR(True)
0201610165 RA (False), PB (False), DIR(True)
0201610174 RA (False), PB (True), DIR(True)
0201610261 RA (False), PB (False), DIR(True)
0201610285 RA 

0401610207 RA (False), PB (False), DIR(True)
0401610264 RA (False), PB (False), DIR(True)
0401610267 RA (False), PB (False), DIR(True)
0401610274 RA (False), PB (False), DIR(True)
0401610294 RA (False), PB (False), DIR(True)
0401610362 RA (False), PB (False), DIR(True)
0401610405 RA (False), PB (True), DIR(True)
0401610574 RA (False), PB (False), DIR(True)
0401610646 RA (False), PB (False), DIR(True)
0401610760 RA (False), PB (False), DIR(True)
0401610783 RA (False), PB (False), DIR(True)
0401610801 RA (False), PB (True), DIR(True)
0401610881 RA (False), PB (False), DIR(True)
0401610892 RA (False), PB (True), DIR(True)
0401610959 RA (False), PB (False), DIR(True)
0401610987 RA (False), PB (False), DIR(True)
0401610988 RA (False), PB (False), DIR(True)
0401611075 RA (False), PB (False), DIR(True)
0401611115 RA (False), PB (False), DIR(True)
0401611136 RA (False), PB (True), DIR(True)
0401611137 RA (False), PB (True), DIR(True)
0401611202 RA (False), PB (False), DIR(True)
0401611239 RA (

0451610369 RA (False), PB (False), DIR(True)
0451610413 RA (False), PB (False), DIR(True)
0451610468 RA (False), PB (False), DIR(True)
0451610500 RA (False), PB (True), DIR(True)
0451610550 RA (False), PB (False), DIR(True)
0451610566 RA (False), PB (False), DIR(True)
0451610664 RA (False), PB (False), DIR(True)
0451610709 RA (False), PB (False), DIR(True)
0451610717 RA (False), PB (False), DIR(True)
0451610769 RA (False), PB (False), DIR(True)
0451610774 RA (False), PB (False), DIR(True)
0451610800 RA (False), PB (False), DIR(True)
0451610841 RA (False), PB (False), DIR(True)
0451610899 RA (False), PB (False), DIR(True)
0451611077 RA (False), PB (False), DIR(True)
0451611187 RA (False), PB (False), DIR(True)
0451611238 RA (False), PB (False), DIR(True)
0451611350 RA (False), PB (False), DIR(True)
0451611381 RA (False), PB (False), DIR(True)
0451611416 RA (False), PB (False), DIR(True)
0451611429 RA (False), PB (False), DIR(True)
0451611536 RA (False), PB (False), DIR(True)
0451611933 

...

1051613053 RA (False), PB (False), DIR(True)
1051613102 RA (False), PB (False), DIR(True)
1051613113 RA (False), PB (False), DIR(True)
1051613114 RA (False), PB (False), DIR(True)
1051613121 RA (False), PB (False), DIR(True)
1051613133 RA (False), PB (False), DIR(True)
1051613151 RA (False), PB (False), DIR(True)
1051613209 RA (False), PB (False), DIR(True)
1051613231 RA (False), PB (False), DIR(True)
1051613344 RA (False), PB (False), DIR(True)
1051613349 RA (False), PB (False), DIR(True)
1051613480 RA (False), PB (True), DIR(True)
1051613490 RA (False), PB (False), DIR(True)
1051613547 RA (False), PB (True), DIR(True)
1051613590 RA (False), PB (False), DIR(True)
1051613610 RA (False), PB (False), DIR(True)
1051613638 RA (False), PB (False), DIR(True)
1051613667 RA (False), PB (False), DIR(True)
1051613682 RA (False), PB (False), DIR(True)
1051613752 RA (False), PB (False), DIR(True)
1051613761 RA (False), PB (False), DIR(True)
1051613813 RA (False), PB (False), DIR(True)
1051613816 R

1081612241 RA (False), PB (False), DIR(True)
1081612369 RA (False), PB (False), DIR(True)
1081612382 RA (False), PB (False), DIR(True)
1081612503 RA (False), PB (False), DIR(True)
1081612506 RA (False), PB (True), DIR(True)
1081612529 RA (False), PB (False), DIR(True)
1081612538 RA (False), PB (True), DIR(True)
1081612545 RA (False), PB (False), DIR(True)
1081612623 RA (False), PB (True), DIR(True)
1081612623 RA (False), PB (False), DIR(True)
1081612651 RA (False), PB (False), DIR(True)
1081612735 RA (False), PB (True), DIR(True)
1081612812 RA (False), PB (False), DIR(True)
1081612824 RA (False), PB (False), DIR(True)
1081612869 RA (False), PB (False), DIR(True)
1081612873 RA (False), PB (False), DIR(True)
1081612875 RA (False), PB (True), DIR(True)
1081612882 RA (False), PB (False), DIR(True)
1081612971 RA (False), PB (True), DIR(True)
1091610021 RA (False), PB (False), DIR(True)
1091610051 RA (False), PB (False), DIR(True)
1091610079 RA (False), PB (False), DIR(True)
1091610125 RA (F

1101612163 RA (False), PB (False), DIR(True)
1101612164 RA (False), PB (True), DIR(True)
1101612175 RA (False), PB (False), DIR(True)
1101612179 RA (False), PB (False), DIR(True)
1101612198 RA (False), PB (False), DIR(True)
1101612214 RA (False), PB (False), DIR(True)
1101612226 RA (False), PB (True), DIR(True)
1111610051 RA (False), PB (False), DIR(True)
1111610100 RA (False), PB (False), DIR(True)
1111610121 RA (False), PB (False), DIR(True)
1111610193 RA (False), PB (False), DIR(True)
1111610315 RA (False), PB (False), DIR(True)
1111610320 RA (False), PB (False), DIR(True)
1111610344 RA (False), PB (False), DIR(True)
1111610445 RA (False), PB (False), DIR(True)
1111610449 RA (False), PB (False), DIR(True)
1111610555 RA (False), PB (False), DIR(True)
1111610626 RA (False), PB (True), DIR(True)
1111610669 RA (False), PB (False), DIR(True)
1111610757 RA (False), PB (False), DIR(True)
1111610792 RA (False), PB (False), DIR(True)
1111610810 RA (False), PB (False), DIR(True)
1111610842 RA

1141610154 RA (False), PB (True), DIR(True)
1141610219 RA (False), PB (False), DIR(True)
1141610244 RA (False), PB (False), DIR(True)
1141610251 RA (False), PB (False), DIR(True)
1141610258 RA (False), PB (True), DIR(True)
1141610271 RA (False), PB (False), DIR(True)
1141610296 RA (False), PB (False), DIR(True)
1141610308 RA (False), PB (False), DIR(True)
1141610326 RA (False), PB (False), DIR(True)
1141610345 RA (False), PB (False), DIR(True)
1141610429 RA (False), PB (False), DIR(True)
1141610516 RA (False), PB (False), DIR(True)
1141610550 RA (False), PB (False), DIR(True)
1141610573 RA (False), PB (False), DIR(True)
1141610581 RA (False), PB (False), DIR(True)
1141610615 RA (False), PB (False), DIR(True)
1141610705 RA (False), PB (False), DIR(True)
1141610724 RA (False), PB (False), DIR(True)
1141610736 RA (False), PB (False), DIR(True)
1141610805 RA (False), PB (False), DIR(True)
1141610808 RA (False), PB (False), DIR(True)
1141610828 RA (False), PB (False), DIR(True)
1141610832 R

1211610432 RA (False), PB (False), DIR(True)
1211610438 RA (False), PB (False), DIR(True)
1211610454 RA (False), PB (False), DIR(True)
1211610525 RA (False), PB (False), DIR(True)
1211610528 RA (False), PB (False), DIR(True)
1211610567 RA (False), PB (False), DIR(True)
1211610599 RA (False), PB (False), DIR(True)
1211610716 RA (False), PB (False), DIR(True)
1211610815 RA (False), PB (False), DIR(True)
1211610839 RA (False), PB (False), DIR(True)
1211610846 RA (False), PB (False), DIR(True)
1211610849 RA (False), PB (False), DIR(True)
1211610924 RA (False), PB (False), DIR(True)
1211610938 RA (False), PB (False), DIR(True)
1211610947 RA (False), PB (False), DIR(True)
1211610987 RA (False), PB (False), DIR(True)
1211611000 RA (False), PB (False), DIR(True)
1211611038 RA (False), PB (False), DIR(True)
1211611083 RA (False), PB (False), DIR(True)
1211611120 RA (False), PB (False), DIR(True)
1211611166 RA (False), PB (False), DIR(True)
1211611176 RA (False), PB (False), DIR(True)
1211611182

In [25]:
def approved_intersections(md):
    ints_w_warrant_met = dict()
    for master in md.keys():
        if md[master][-1].warrant():
            ints_w_warrant_met[master] = md[master][-1].get_12_month_crashes()
            #print master, len(md[master][-1].twelve_month_period())
    return ints_w_warrant_met

In [26]:
approved_masters = approved_intersections(masters)

1231610296 RA (False), PB (False), DIR(True)
1231610036 RA (False), PB (False), DIR(True)
1231610599 RA (False), PB (False), DIR(True)
1231610915 RA (False), PB (False), DIR(True)
1231610389 RA (False), PB (False), DIR(True)
0601610553 RA (False), PB (False), DIR(True)
0601610584 RA (False), PB (False), DIR(True)
1231610768 RA (False), PB (False), DIR(True)
1231610830 RA (False), PB (False), DIR(True)
1201611062 RA (False), PB (False), DIR(True)
1231610497 RA (False), PB (False), DIR(True)
1231610095 RA (False), PB (False), DIR(True)
0881610153 RA (False), PB (True), DIR(True)
1231611059 RA (False), PB (False), DIR(True)
0751612537 RA (False), PB (False), DIR(True)
0751612587 RA (False), PB (True), DIR(True)
1231610824 RA (False), PB (False), DIR(True)
1231610824 RA (False), PB (False), DIR(True)
1231610788 RA (False), PB (False), DIR(True)
1231610727 RA (False), PB (False), DIR(True)
1231610670 RA (False), PB (False), DIR(True)
1231610074 RA (False), PB (False), DIR(True)
1231610079 R

0701610582 RA (False), PB (False), DIR(True)
0701610193 RA (False), PB (False), DIR(True)
0701611648 RA (False), PB (False), DIR(True)
0701610998 RA (False), PB (False), DIR(True)
0611611212 RA (False), PB (False), DIR(True)
0611610113 RA (False), PB (False), DIR(True)
0631611181 RA (False), PB (False), DIR(True)
0631610286 RA (False), PB (False), DIR(True)
0661610944 RA (False), PB (False), DIR(True)
0661610872 RA (False), PB (False), DIR(True)
0661610872 RA (False), PB (False), DIR(True)
0661611248 RA (False), PB (False), DIR(True)
0661610136 RA (False), PB (True), DIR(True)
0701610447 RA (False), PB (False), DIR(True)
0701611242 RA (False), PB (False), DIR(True)
0701611242 RA (False), PB (False), DIR(True)
0661611670 RA (False), PB (True), DIR(True)
0661611670 RA (False), PB (True), DIR(True)
0701610553 RA (False), PB (True), DIR(True)
0701611304 RA (False), PB (True), DIR(True)
0701610108 RA (False), PB (False), DIR(True)
0701611368 RA (False), PB (False), DIR(True)
0701611906 RA (

0171612321 RA (False), PB (True), DIR(True)
0171610700 RA (False), PB (False), DIR(True)
0181611371 RA (False), PB (False), DIR(True)
0181612785 RA (False), PB (False), DIR(True)
0181612391 RA (False), PB (False), DIR(True)
0171612019 RA (False), PB (False), DIR(True)
0171612181 RA (False), PB (False), DIR(True)
0171610284 RA (False), PB (False), DIR(True)
0171610570 RA (False), PB (False), DIR(True)
0171610856 RA (False), PB (False), DIR(True)
0181611744 RA (False), PB (False), DIR(True)
0171610920 RA (False), PB (False), DIR(True)
0171611363 RA (False), PB (False), DIR(True)
0171610920 RA (False), PB (False), DIR(True)
0171611363 RA (False), PB (False), DIR(True)
0181612488 RA (False), PB (True), DIR(True)
0171610861 RA (False), PB (False), DIR(True)
0181612159 RA (False), PB (False), DIR(True)
0181610624 RA (False), PB (False), DIR(True)
0191613368 RA (False), PB (False), DIR(True)
0191611969 RA (False), PB (True), DIR(True)
0191601049 RA (False), PB (True), DIR(True)
0171610744 RA 

1041611184 RA (False), PB (False), DIR(True)
1041610341 RA (False), PB (False), DIR(True)
1041610126 RA (False), PB (False), DIR(True)
1041610126 RA (False), PB (False), DIR(True)
1041612511 RA (False), PB (False), DIR(True)
1041612334 RA (False), PB (False), DIR(True)
1041610819 RA (False), PB (False), DIR(True)
1041611167 RA (False), PB (False), DIR(True)
1041610926 RA (False), PB (False), DIR(True)
1041610854 RA (False), PB (True), DIR(True)
1041611893 RA (False), PB (True), DIR(True)
1041610756 RA (False), PB (False), DIR(True)
1041611593 RA (False), PB (False), DIR(True)
1041610847 RA (False), PB (False), DIR(True)
1041611180 RA (False), PB (False), DIR(True)
1041611180 RA (False), PB (False), DIR(True)
1041610900 RA (False), PB (False), DIR(True)
1041610328 RA (False), PB (False), DIR(True)
1041610649 RA (False), PB (False), DIR(True)
1041611751 RA (False), PB (False), DIR(True)
1041612077 RA (False), PB (False), DIR(True)
1041610647 RA (False), PB (False), DIR(True)
1041611874 R

0251610881 RA (False), PB (True), DIR(True)
0251610834 RA (False), PB (False), DIR(True)
0251610834 RA (False), PB (False), DIR(True)
0321610133 RA (False), PB (True), DIR(True)
0321610548 RA (False), PB (False), DIR(True)
0321610409 RA (False), PB (False), DIR(True)
0321610318 RA (False), PB (False), DIR(True)
0321610811 RA (False), PB (True), DIR(True)
0321610318 RA (False), PB (False), DIR(True)
0321610811 RA (False), PB (True), DIR(True)
0321610926 RA (False), PB (False), DIR(True)
0251611758 RA (False), PB (False), DIR(True)
0251611758 RA (False), PB (False), DIR(True)
0251610917 RA (False), PB (False), DIR(True)
0401611304 RA (False), PB (False), DIR(True)
0401612034 RA (False), PB (False), DIR(True)
0401612045 RA (False), PB (False), DIR(True)
0401611304 RA (False), PB (False), DIR(True)
0401612034 RA (False), PB (False), DIR(True)
0401612045 RA (False), PB (False), DIR(True)
0401610207 RA (False), PB (False), DIR(True)
0401611416 RA (False), PB (True), DIR(True)
0401610783 RA (

0471611860 RA (False), PB (False), DIR(True)
0471612172 RA (False), PB (False), DIR(True)
0471611233 RA (False), PB (False), DIR(True)
0471612488 RA (False), PB (False), DIR(True)
0471611507 RA (False), PB (False), DIR(True)
0471610363 RA (False), PB (False), DIR(True)
0471610891 RA (False), PB (False), DIR(True)
0471611351 RA (False), PB (False), DIR(True)
0471612028 RA (False), PB (False), DIR(True)
0471612310 RA (False), PB (False), DIR(True)
0471611351 RA (False), PB (False), DIR(True)
0471612028 RA (False), PB (False), DIR(True)
0471612310 RA (False), PB (False), DIR(True)
0471610534 RA (False), PB (False), DIR(True)
0471612456 RA (False), PB (False), DIR(True)
0471611142 RA (False), PB (False), DIR(True)
0471612493 RA (False), PB (False), DIR(True)
0471611024 RA (False), PB (False), DIR(True)
0471610072 RA (False), PB (False), DIR(True)
0471610072 RA (False), PB (False), DIR(True)
0471610704 RA (False), PB (False), DIR(True)
0471610582 RA (False), PB (False), DIR(True)
0471611444

1031610225 RA (False), PB (False), DIR(True)
1211611000 RA (False), PB (False), DIR(True)


In [27]:
print(approved_masters)

{6147: {u'1211610947': <__main__.Crash instance at 0x0C3214B8>, u'1211610921': <__main__.Crash instance at 0x0C3212B0>, u'1211610281': <__main__.Crash instance at 0x0C313990>, u'1211610836': <__main__.Crash instance at 0x0C31CD78>, u'1211610599': <__main__.Crash instance at 0x0C318F58>}, 20485: {u'0051610232': <__main__.Crash instance at 0x0A2CE558>, u'0051611562': <__main__.Crash instance at 0x0A3731E8>, u'0051610066': <__main__.Crash instance at 0x0A2CA6C0>, u'0051611345': <__main__.Crash instance at 0x0A2EA080>, u'0051611545': <__main__.Crash instance at 0x0A373080>, u'0051610464': <__main__.Crash instance at 0x0A2D3940>}, 20489: {u'0051610813': <__main__.Crash instance at 0x0A2DC5D0>, u'0051610740': <__main__.Crash instance at 0x0A2DC030>, u'0051610637': <__main__.Crash instance at 0x0A2D7738>, u'0051610969': <__main__.Crash instance at 0x0A2E02D8>, u'0051610577': <__main__.Crash instance at 0x0A2D7238>, u'0051610679': <__main__.Crash instance at 0x0A2D7B48>, u'0051611030': <__main

In [28]:
def get_node_details(pg):
    data = dict()
    cur = pg.conn.cursor()
    cur.execute("""select  nodeid, masterid, st1, st2, st_x(geom) as x, st_y(geom) as y
                    from node n
                    join (
                        select nodeidfrom, min(st1) as st1, max(st2) as st2
                        from(
                            select s1.nodeidfrom, s1.street as st1, s2.street  as st2
                            from lion as s1 join lion as s2
                            on s1.nodeidfrom = s2.nodeidto
                            where s1.street != s2.street
                            and s1.featuretyp not in ('1', '2', '3', '7')
                            and s2.featuretyp not in ('1', '2', '3', '7')
                        ) as names_ group by nodeidfrom
                    ) as st_names on n.nodeid::int = st_names.nodeidfrom::int
                """)
    for row in cur.fetchall():
        nodeid, masterid, st1, st2, x, y = row
        data[nodeid] = [st1, st2, x, y]
    del cur
    return data

In [29]:
node_data = get_node_details(pgo)
node_data

{Decimal('9043971'): ['CEDAR AVENUE',
  'SEDGWICK AVENUE',
  1006251.0139955,
  249834.141871974],
 Decimal('9043976'): ['DRIVEWAY',
  'CLIFTON AVENUE',
  963095.125501141,
  161331.486815557],
 Decimal('10'): ['WARDS POINT AVENUE',
  'HYLAN BOULEVARD',
  913671.212089136,
  122413.678241149],
 Decimal('11'): ['CANTER AVENUE',
  'HYLAN BOULEVARD',
  913966.704595372,
  122458.842518911],
 Decimal('14'): ['WARDS POINT AVENUE',
  'PERTH AMBOY PLACE',
  913553.959376037,
  123788.918540359],
 Decimal('9043983'): ['ALLEY', 'NEW LANE', 966923.552715108, 162602.589860439],
 Decimal('18'): ['SATTERLEE STREET',
  'SHORE ROAD',
  913844.414819092,
  123550.700851426],
 Decimal('19'): ['PERTH AMBOY PLACE',
  'SATTERLEE STREET',
  913868.315688878,
  123836.428285763],
 Decimal('22'): ['WARDS POINT AVENUE',
  'TOTTENVILLE PLACE',
  913469.290913939,
  124077.418606833],
 Decimal('23'): ['AVIVA COURT',
  'AMBOY ROAD',
  913353.156964973,
  124359.711008817],
 Decimal('25'): ['AMBOY ROAD',
  'HOPPI

In [56]:
for i in node_data:
    if i == 33755:
     print node_data[i]

['ROCKAWAY BOULEVARD', '110 STREET', 1031726.09807865, 186224.401813105]


In [58]:
def intersection_summaries(approved, mids, nds):
    summary = list()
    for m in approved.keys():
        crashes = len([c for c in approved[m]])
        ped_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 1])
        bike_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 2])
        mvo_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 3])
        right_angles = len([approved[m][c].crashid for c in approved[m] if approved[m][c].col_typ in ('4', '04')])
        date_ranges = [min([approved[m][c].ac_date for c in approved[m]]),
                       max([approved[m][c].ac_date for c in approved[m]])]
        print 'Master %i (%s - %s):\n\tNodes %s \n\t%i crashes\n\t%i ped injuries\n\t%i bike' \
              'injuries\n\t%i mvo injuries\n\t%i right angle crashes' % (m, date_ranges[0].strftime('%m/%d/%Y'),
                                                                         date_ranges[1].strftime('%m/%d/%Y'),
                                                                         str(mids[m][-1].nodeid_list),
                                                                         crashes, ped_inj, bike_inj, mvo_inj,
                                                                         right_angles)
        if len(mids[m]) > 2:
            for node in mids[m][:-1]:
                if node in nds.keys():
                    s1, s2, x, y = nds[node]
                else:
                    s1, s2, x, y = '', '', 0, 0
                summary.append([m, node, crashes, ped_inj, bike_inj, mvo_inj, ped_inj+bike_inj+mvo_inj,
                                right_angles, date_ranges[0], date_ranges[1], s1, s2, x, y])
        else:
            if mids[m][0] in nds.keys():
                s1, s2, x, y = nds[mids[m][0]]
            else:
                s1, s2, x, y = '', '', 0, 0
            summary.append([m, mids[m][0], crashes, ped_inj, bike_inj, mvo_inj, ped_inj+bike_inj+mvo_inj,
                            right_angles, date_ranges[0], date_ranges[1], s1, s2, x, y])
    return ['MasterID', 'NodeID', 'Crashes', 'Ped_Injs', 'Bike_Injs', 'MVO_Injs', 'Total_Injs',
            'Right_Angle_Crashes', 'Earliest_Date', 'Latest_Date', 'Street1', 'Street2', 'X', 'Y'], summary

In [59]:
headers, crash_data = intersection_summaries(approved_masters, masters, node_data)


Master 33755 (03/24/2016 - 10/16/2016):
	Nodes [33755] 
	12 crashes
	3 ped injuries
	0 bikeinjuries
	3 mvo injuries
	7 right angle crashes


In [32]:
for i in range(5):
    print headers[]


SyntaxError: invalid syntax (<ipython-input-32-e67a03377f2b>, line 2)

In [48]:
print headers[1:14]

['NodeID', 'Crashes', 'Ped_Injs', 'Bike_Injs', 'MVO_Injs', 'Total_Injs', 'Right_Angle_Crashes', 'Earliest_Date', 'Latest_Date', 'Street1', 'Street2', 'X', 'Y']


In [25]:
print approved_masters.keys()

[33755]


In [28]:
print len(approved_masters[33755])

12
